In [3]:
import cv2
import face_recognition
import numpy as np
import os
from datetime import datetime
import pandas as pd

# Path to the folder containing known face images
path = 'ImagesAttendance'
images = []
classNames = []
myList = os.listdir(path)

# Loop through each image in the directory and load it
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

# Function to find encodings for all images
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

# Get encodings of all known faces
encodeListKnown = findEncodings(images)

# Create or update attendance in a single file
def markAttendance(name):
    # The same file for all attendance records
    file_name = 'Attendance GR10.csv'
    
    # Get the current time and date
    current_time = datetime.now().strftime('%H:%M:%S')
    current_date = datetime.now().strftime('%Y-%m-%d')
    
    # Load or create the attendance DataFrame
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
        # Ensure DataFrame has the correct columns
        if 'Name' not in df.columns or 'Date' not in df.columns or 'Time' not in df.columns:
            df = pd.DataFrame(columns=['Name', 'Date', 'Time'])
    else:
        df = pd.DataFrame(columns=['Name', 'Date', 'Time'])
    
    # Check if the name and date are already marked to avoid duplicates
    if not ((df['Name'] == name) & (df['Date'] == current_date)).any():
        new_row = pd.DataFrame({'Name': [name], 'Date': [current_date], 'Time': [current_time]})
        df = pd.concat([df, new_row], ignore_index=True)
        df.to_csv(file_name, index=False)

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    
    if not success:
        print("Error: Failed to capture image")
        break
    
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            markAttendance(name)

    cv2.imshow('Webcam', img)
    
    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
